In [54]:
import sys , os 
import numpy as np
import math as m
import pandas as pd
import csv



In [64]:
energy='12'
heter='b'
depth='30'

In [65]:
mesurePlot="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/"+energy+"E/new/f"+heter+energy+"e"+depth+".csv"
gatePlot="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/"+energy+"E/new/g"+heter+energy+"e"+depth+".csv"
emcPlot="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/"+energy+"E/new/emc"+heter+energy+"e"+depth+".csv"

gateuncertaityPlot="/home/crct/CRCT/JupyterNotebook/data/gammaIndex/"+energy+"E/new/uncertainty/"+heter+"g"+energy+"e"+depth+"_uncertainty.csv"

mesurePlotData=pd.read_csv(mesurePlot, sep=',')
gatePlotData=pd.read_csv(gatePlot, sep=',')
emcPlotData=pd.read_csv(emcPlot, sep=',')


gateuncertaintyPlotData=pd.read_csv(gateuncertaityPlot, sep=',')
gateuncertaintyPlotDataw=pd.read_csv(gateuncertaityPlot, sep=',')

In [66]:
LinearInterpolation=True

Threshold=True
ThresholdValue=0.1

#Gamma Index criterion 
DTA=3 #mm
deltaDmax=0.03 #%

name=energy+heter+depth

In [67]:
xGscale=6
xEMCscale=5

mdown=-20
mup=-15

In [68]:
#Create numpy ndarray from pandas dataFrames
refX=mesurePlotData['pas']
refY=mesurePlotData['value']
gateX=gatePlotData['pas']
gateY=gatePlotData['value']
emcX=emcPlotData['pas']
emcY=emcPlotData['value']

gateUx=gateuncertaintyPlotData['pas']
gateUy=gateuncertaintyPlotData['value']


refX=refX-(refX.max()/2)
emcX=emcX-(emcX.max()/2)

#Dose Grid scaling
#emcY=(emcY*4.0e-6)/2
#refY=refY*0.01
#gateUy=gateUy*100

# Compute Linear interpolation for simulated values and measured values according to your parameters (change 1000 values to change number of points)
if LinearInterpolation==True:
    refX2=np.linspace(refX.min(), refX.max(), 2000)
    refY2=np.interp(refX2, refX, refY)
    gateX2=np.linspace(gateX.min(), gateX.max(), 2000)
    gateY2=np.interp(gateX2, gateX, gateY)
    emcX2=np.linspace(emcX.min(), emcX.max(), 2000)
    emcY2=np.interp(emcX2, emcX, emcY)

    gateUx2=np.linspace(gateUx.min(), gateX.max(), 2000)
    gateUy2=np.interp(gateUx2, gateUx, gateUy)

else:
    refX2= refX
    refY2= refY
    gateX2= gateX
    gateY2= gateY
    emcX2= emcX
    emcY2= emcY

    

emcY2=np.flipud(emcY2)

maskFilm=(refX2 > mdown) & (refX2 < mup)
maskGate=(gateX2 > mdown) & (gateX2 < mup)
maskEmc=(emcX2 > mdown) & (emcX2 < mup)

#normalisation mean homogeneity
refY2=refY2/refY2[maskFilm].mean()
gateY2=gateY2/gateY2[maskGate].mean()
emcY2=emcY2/emcY2[maskEmc].mean()

#uncertainies

gateUy2=gateUy2*gateY2

    #SCALING
gateX2=gateX2+xGscale
emcX2=emcX2+xEMCscale
print 'ok'



ok


In [69]:
#  GATE  #
#create numpy ndarray for gamma comprison
deltaD=np.zeros(len(refY2))
gammaIndex=np.zeros(len(gateX2))
gammaIndexTEMP=np.zeros(len(refX2))
drTEMP=np.zeros(len(refX2))
dr=np.zeros(len(refX2))

#compute Gamma Index 1D
for i in range(len(gateX2)):
    for j in range(len(refX2)):
        dr[j]=(gateX2[i]-refX2[j])
        deltaD[j]=gateY2[i]-refY2[j]
        gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/deltaDmax**2) + ((dr[j]**2)/DTA**2))           
    gammaIndex[i]=gammaIndexTEMP.min()

#Compute Gamma Index Pass Rate (with threshold or not)
nok=0.0
a=0
if Threshold==True: 
    for i in range(len(gammaIndex)):

        if gateY2[i]>0.10:
            a=a+1
            if gammaIndex[i]<1:
                nok=nok+1
    print ('nombre de points testés : ' +str(a))
    print ('nombre de points passant le test (<1) : ' +str(nok))
    passRate=(nok/a)*100
else:
    nok=(gammaIndex < 1).sum()
    passRate=nok*100.0/len(gammaIndex)

#print results
percentDmax=deltaDmax*100
print('RESULTAT GATE : ')
print('*   Gamma Index Pass Rate '+str(percentDmax) + '%-' +str(DTA) + 'mm: '  +str(passRate) +' %')

nombre de points testés : 1265
nombre de points passant le test (<1) : 1251.0
RESULTAT GATE : 
*   Gamma Index Pass Rate 3.0%-3mm: 98.8932806324 %


In [70]:
#    EMC   #
#create numpy ndarray for gamma comprison
deltaD=np.zeros(len(refY2))
gammaIndex2=np.zeros(len(emcX2))
gammaIndexTEMP=np.zeros(len(refX2))
drTEMP=np.zeros(len(refX2))
dr=np.zeros(len(refX2))

#compute Gamma Index 1D
for i in range(len(emcX2)):
    for j in range(len(refX2)):
        dr[j]=(emcX2[i]-refX2[j])
        deltaD[j]=emcY2[i]-refY2[j]
        gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/deltaDmax**2) + ((dr[j]**2)/DTA**2))
    #print gammaIndexTEMP.argmin()
    gammaIndex2[i]=gammaIndexTEMP.min()
#print gammaIndex2
#Compute Gamma Index Pass Rate (with threshold or not)
nok=0.0
a=0
if Threshold==True: 
    for i in range(len(gammaIndex2)):

        if emcY2[i]>0.10:
            a=a+1
            if gammaIndex2[i]<1:
                nok=nok+1
    print ('nombre de points testés : ' +str(a))
    print ('nombre de points passant le test (<1) : ' +str(nok))
    passRate2=(nok/a)*100
else:
    nok=(gammaIndex2 < 1).sum()
    passRate2=nok*100.0/len(gammaIndex2)

#print results
percentDmax=deltaDmax*100
print('RESULTAT EMC : ')
print('*   Gamma Index Pass Rate '+str(percentDmax) + '% - ' +str(DTA) + 'mm : '  +str(passRate2) +' %')

nombre de points testés : 861
nombre de points passant le test (<1) : 640.0
RESULTAT EMC : 
*   Gamma Index Pass Rate 3.0% - 3mm : 74.3321718931 %


In [71]:
GIPR={}
GIPR['exp']=name
GIPR['GATE']=passRate
GIPR['EMC']=passRate2

gipr=open('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/'+energy+'E/'+name+'GIPR.txt', 'w')

gipr.write(GIPR['exp'] + '\n')
gipr.write('GATE:' + '\n')
gipr.write(str(GIPR['GATE']) + '\n')
gipr.write('EMC:' + '\n')
gipr.write(str(GIPR['EMC']) + '\n')
gipr.close()


In [72]:
mes=np.column_stack((refX2, refY2))
gate=np.column_stack((gateX2, gateY2))
emc=np.column_stack((emcX2, emcY2))
gamma1=np.column_stack((gateX2, gammaIndex))
gamma2=np.column_stack((emcX2, gammaIndex2))

np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/'+energy+'E/mes_'+name+'.csv', mes, delimiter=',')
np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/'+energy+'E/gate_'+name+'.csv', gate, delimiter=',')
np.savetxt('/home/crct/CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/'+energy+'E/emc_'+name+'.csv', emc, delimiter=',')
np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/'+energy+'E/gammaGate_'+name+'.csv', gamma1, delimiter=',')
np.savetxt('/media/sf_CRCT/JupyterNotebook/data/plot/gamaIndex/elecArticle/'+energy+'E/gammaEmc_'+name+'.csv', gamma2, delimiter=',')
